### Read best model

In [8]:
import coremlv2 as core
import importlib
# core._init_ml()
importlib.reload(core)

<module 'coremlv2' from 'C:\\Users\\jonat\\Documents\\#PROJECT\\idx\\coremlv2.py'>

In [2]:
ticker_group = ['wsd']
kt_iter = '24'
print(core.read_best_model_hyperparameters_name(ticker_group, kt_iter))

['superblock_final_dunits', 'lr', 'comp2', 'comp3', 'comp5', 'comp7', 'comp11', 'comp13', 'comp17', 'comp19', 'comp23', 'comp29', 'comp31', 'comp37', 'comp41', 'comp43', 'comp47', 'comp53', 'comp59', 'comp61', 'comp67', 'comp71', 'comp73', 'comp79', 'comp83', 'comp89', 'comp97', 'comp101', 'comp103', 'comp107', 'comp109', 'comp113', 'tuner/epochs', 'tuner/initial_epoch', 'tuner/bracket', 'tuner/round']


### read_retrain_model
- derived from read_best_model, but from whole data training (different folder structure)

In [1]:
import coremlv2 as core
import importlib
core._init_ml()
importlib.reload(core)
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [3]:
ROOT_PATH = './'
# kt_iter = '9'
# retrain_epochs = 1
# source_group_name = 'MERGED-shuffled'
# source_model_path = core.os.path.join(ROOT_PATH, f'models/kt/v{kt_iter}/{source_group_name}_retrain-epochs{retrain_epochs}/model')

# source_model = core.tf.keras.models.load_model(source_model_path)

model_base_id = '307'
model_source = ['BBCA']
model_ticker_target = ['BBCA']
shift = 0
interval = 1
recurrent = 120
db_ver = '3'
dataset_ver = '4'
kt_iter = '9'
split = 0.8

model, data_version = core.read_best_model(model_base_id, model_source, model_ticker_target, shift, interval, recurrent, db_ver, dataset_ver, kt_iter, split, ROOT_PATH, retrain=False, backtest=False, save_model=False, generator=True)


Load weights from epoch 2


In [ ]:
ROOT_PATH = './'
# kt_iter = '9'
# retrain_epochs = 1
# source_group_name = 'MERGED-shuffled'
# source_model_path = core.os.path.join(ROOT_PATH, f'models/kt/v{kt_iter}/{source_group_name}_retrain-epochs{retrain_epochs}/model')

# source_model = core.tf.keras.models.load_model(source_model_path)

model_base_id = '310'
model_source = ['MERGED-shuffled']
model_ticker_target = ['MERGED-shuffled']
shift = 0
interval = 1
recurrent = 120
db_ver = '4'
dataset_ver = '1'
kt_iter = '12'
split = 0.8

model, data_version = core.read_best_model(model_base_id, model_source, model_ticker_target, shift, interval, recurrent, db_ver, dataset_ver, kt_iter, split, ROOT_PATH, retrain=False, backtest=False, save_model=False, generator=True)


In [20]:
# Load models without retraining v2
# def load_model()
ROOT_PATH = './'
kt_iter = '33.old (80 20 traintest slice)'
model_no = '326'
base_path = core.os.path.join(ROOT_PATH, 'models')
base_path = core.os.path.join(base_path, 'kt')
base_path = core.os.path.join(base_path, f'v{kt_iter}')

last_epochs = core.last_epochs_from_directory(base_path, 'wsd')

# Load model backbone
model_source_path = core.os.path.join(base_path, 'wsd')
best_model = core.best_model_metadata(model_source_path, metrics=['val_loss','val_accuracy'], primary_metrics='val_loss', ascending=True)
modelv2 = core.model_switcher_kt_build(model_no, core.hp_params(best_model), 0, 0)

if last_epochs == 0:
    print('Load model from keras_tuner')
    # Load weights from keras_tuner directory
    weights_path = core.os.path.join(model_source_path, f'{best_model.id}')
    weights_path = core.os.path.join(weights_path, 'checkpoints')
    weights_path = core.os.path.join(weights_path, 'epoch_0')
    weights_path = core.os.path.join(weights_path, 'checkpoint')
    modelv2.load_weights(weights_path)
    # If have been trained before, continue training for last epoch
elif last_epochs > 0:
    print(f'Load weights from epoch {last_epochs}')
    # Load weights from previously trained model
    weights_path = core.os.path.join(base_path, f'wsd_retrain-epochs{last_epochs}')
    weights_path = core.os.path.join(weights_path, 'weights')
    weights_path = core.os.path.join(weights_path, 'checkpoint')
    modelv2.load_weights(weights_path)

Load model from keras_tuner


In [2]:
# Load model (not weight) v2
ROOT_PATH = './'
kt_iter = '33.old (80 20 traintest slice)'
base_path = core.os.path.join(ROOT_PATH, 'models')
base_path = core.os.path.join(base_path, 'kt')
base_path = core.os.path.join(base_path, f'v{kt_iter}')

last_epochs = core.last_epochs_from_directory(base_path, 'wsd')

# Load model backbone
if last_epochs > 0:
    model_source_path = core.os.path.join(base_path, f'wsd_retrain-epochs{last_epochs}')
    model_source_path = core.os.path.join(model_source_path, 'model')
    modelv3 = core.tf.keras.models.load_model(model_source_path)
elif last_epochs == 0:
    print('Retrain model first before use.')

In [ ]:
target_group = ['BBCA']
target_group_name = '_'.join(target_group)
target_model_path = core.os.path.join(ROOT_PATH, f'models/kt/v{kt_iter}/{source_group_name}_epochs-multiplier{source_epochs_multiplier}/retrained/{target_group_name}')

# Check if current target retrained model configurations have been tried before
if core.os.path.exists(core.os.path.join(target_model_path, 'models/')):
    # Directly read retrained model
    model = core.tf.keras.models.load_model(core.os.path.join(target_model_path, 'models/'))
else:
    # Read whole data trained model

    # Load dataset for retraining
    shift = 0
    interval = 1
    recurrent = 120
    db_ver = '3'
    dataset_ver = '4'
    split = 0.8
    generator = False
    batch_size = 32
    train_inputs, train_labels, train_changes, test_inputs, test_labels, test_changes, data_version = core.load_dataset(target_group_name, shift, interval, recurrent, db_ver, dataset_ver, split, ROOT_PATH, generator=generator, batch_size=batch_size)

    # Eliminate nan
    train_inputs = np.nan_to_num(train_inputs, posinf=0.0, neginf=0.0)
    test_inputs = np.nan_to_num(test_inputs, posinf=0.0, neginf=0.0)

    # Model retraining configurations
    # Freeze all layers except two final layers.
    epochs = 1
    # model.fit

    # Save model
    model.save(core.os.path.join(target_model_path, 'models/'))
    model.save_weights(core.os.path.join(target_model_path, 'weights/'))
    





In [30]:
for layer in source_model.layers:
    print(layer.name)

input_2
conv1d_531
conv1d_522
conv1d_513
conv1d_504
conv1d_495
conv1d_486
conv1d_477
conv1d_468
conv1d_459
conv1d_450
conv1d_441
conv1d_432
conv1d_423
conv1d_414
conv1d_405
conv1d_396
conv1d_387
conv1d_378
conv1d_369
conv1d_360
conv1d_351
conv1d_342
conv1d_333
conv1d_324
conv1d_315
conv1d_306
conv1d_297
conv1d_288
conv1d_279
conv1d_270
batch_normalization_767
batch_normalization_754
batch_normalization_741
batch_normalization_728
batch_normalization_715
batch_normalization_702
batch_normalization_689
batch_normalization_676
batch_normalization_663
batch_normalization_650
batch_normalization_637
batch_normalization_624
batch_normalization_611
batch_normalization_598
batch_normalization_585
batch_normalization_572
batch_normalization_559
batch_normalization_546
batch_normalization_533
batch_normalization_520
batch_normalization_507
batch_normalization_494
batch_normalization_481
batch_normalization_468
batch_normalization_455
batch_normalization_442
batch_normalization_429
batch_normaliz

In [22]:
target_group = ['BBCA']
target_group_name = '_'.join(target_group)

# Load dataset for retraining
shift = 0
interval = 1
recurrent = 120
db_ver = '3'
dataset_ver = '4'
split = 0.8
generator = False
batch_size = 32
train_inputs, train_labels, train_changes, test_inputs, test_labels, test_changes, data_version = core.load_dataset(target_group, shift, interval, recurrent, db_ver, dataset_ver, split, ROOT_PATH, generator=generator, batch_size=batch_size)

# Eliminate nan
# train_inputs = core.np.nan_to_num(train_inputs, posinf=0.0, neginf=0.0)
# test_inputs = core.np.nan_to_num(test_inputs, posinf=0.0, neginf=0.0)

In [15]:
# OPEN MODEL (whole-data-traning_11, epochs-subepochs)
# 0.4 vid_constituents
hotstart_from = '327-3_cl-0.5_idc-1_mid-0.4_ep-5_bz-1024_ldtv-2_sbuffs-1_gen-1_1-4'

# 0.5 vid_constituents
hotstart_from = '327-3_cl-0.5_idc-1_mid-0.5_ep-5_bz-2048_ldtv-2_sbuffs-1_gen-1_4-16'
hotstart_from = '327-3_cl-0.5_idc-1_mid-0.5_ep-5_bz-2048_ldtv-2_sbuffs-1_gen-1_4-19'

ROOT_PATH = './'

hotstart_save_path = core.os.path.join(ROOT_PATH, f'models/preloaded/{hotstart_from}')
model = core.tf.keras.models.load_model(f'{hotstart_save_path}/model')

In [3]:
# Safe dataset list
dataset = 'vid'
db_ver = '8'
DB_ROOT_PATH = 'E:\#PROJECT\idx'
model_no = '327'
sdl = core.safe_dataset_list(dataset, db_ver=db_ver, DB_ROOT_PATH=DB_ROOT_PATH, return_full_filename=True, model_no=model_no)

In [4]:
sdl[:10]

['AALI.JK_shift0_interval5_recurrent120_split0.8.hdf5',
 'ABMM.JK_shift0_interval5_recurrent120_split0.8.hdf5',
 'ACES.JK_shift0_interval5_recurrent120_split0.8.hdf5',
 'ACST.JK_shift0_interval5_recurrent120_split0.8.hdf5',
 'ADES.JK_shift0_interval5_recurrent120_split0.8.hdf5',
 'ADHI.JK_shift0_interval5_recurrent120_split0.8.hdf5',
 'ADMF.JK_shift0_interval5_recurrent120_split0.8.hdf5',
 'ADRO.JK_shift0_interval5_recurrent120_split0.8.hdf5',
 'AGII.JK_shift0_interval5_recurrent120_split0.8.hdf5',
 'AGRO.JK_shift0_interval5_recurrent120_split0.8.hdf5']

In [46]:
# Load v2 dataset to test wheter the model is valid or not with newer db version
DB_ROOT_PATH = 'E:\\#PROJECT\\idx'
db_path = core.os.path.join(DB_ROOT_PATH, 'db')
db_path = core.os.path.join(db_path, 'v8')
db_path = core.os.path.join(db_path, 'dataset')
db_path = core.os.path.join(db_path, 'vid')
db_path = core.os.path.join(db_path, 'INDF.JK_shift0_interval5_recurrent120_split0.8.hdf5')
# db_path = 'J:\#PROJECT\idx\db\v8\dataset\vid\BBCA.JK_shift0_interval5_recurrent120_split0.8.hdf5'
with core.h5py.File(db_path, 'r') as f:
    train_inputs = f['train_inputs'][()]
    test_inputs = f['test_inputs'][()]
    train_labels = f['train_labels'][()]
    train_changes = f['train_changes'][()]
    test_labels = f['test_labels'][()]
    test_changes = f['test_changes'][()]
    
train_inputs = core.np.nan_to_num(train_inputs, posinf=0.0, neginf=0.0)
test_inputs = core.np.nan_to_num(test_inputs, posinf=0.0, neginf=0.0)

In [47]:
evaluate_train = model.evaluate(train_inputs, train_labels, verbose=1)
evaluate_test = model.evaluate(test_inputs, test_labels, verbose=1)
print(f'Evaluation results: \nTRAIN\nLoss: {evaluate_train[0]:.3f}, accuracy: {evaluate_train[1]:.3f}\nTEST\nLoss: {evaluate_test[0]:.3f}, accuracy: {evaluate_test[1]:.3f}')

print(f'{evaluate_train[0]:.4f} {evaluate_train[1]:.4f} {evaluate_test[0]:.4f} {evaluate_test[1]:.4f}')

24/24 [==============================] - 5s 204ms/step - loss: 0.6928 - accuracy: 0.5645
Evaluation results: 
TRAIN
Loss: 0.611, accuracy: 0.672
TEST
Loss: 0.693, accuracy: 0.564
0.6107 0.6721 0.6928 0.5645


In [48]:
# print(f'Best model metadata:\n', best_model)
train_ideal, train_real, test_ideal, test_real = core.backtest_v1(model, train_inputs, test_inputs, train_labels, test_labels, train_changes, test_changes)
print('Train ideal: ', train_ideal[0], '\nTrain real: ', train_real[0], '\nTest ideal: ', test_ideal[0], '\nTest real: ', test_real[0])
train_r, train_nr, test_r, test_nr = core.performance_ratio(train_ideal, train_real, test_ideal, test_real)
print(f'\nModel backtest performance: tr {train_r:.2f}, tnr {train_nr:.2f}, vr {test_r:.2f}, vnr {test_nr:.2f}')
print('above results is in performance fraction between ideal (observed) and real condition produced from the model\n')

Train ideal:  [ 43.25730415 -26.21570061] 
Train real:  [19.85126233 -2.80965879] 
Test ideal:  [ 6.69984972 -6.52452376] 
Test real:  [ 0.92507786 -0.7497519 ]

Model backtest performance: tr 0.46, tnr 0.11, vr 0.14, vnr 0.11
above results is in performance fraction between ideal (observed) and real condition produced from the model



In [36]:
model.count_params()

927074

In [18]:
modelv3.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 120, 60)]         0         
                                                                 
 lstm (LSTM)                 (None, 112)               77504     
                                                                 
 batch_normalization (BatchN  (None, 112)              448       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 112)               0         
                                                                 
 dense (Dense)               (None, 112)               12656     
                                                                 
 batch_normalization_1 (Batc  (None, 112)              448       
 hNormalization)                                             

### Count and negate

In [197]:
import numpy as np
full_page = np.arange(1,101,1)
laminating = [1,2,3,4,5,6,7,8,9,10,37,38,39,40,46,47,48,49,50,51,52, 61,62,63,64]
non_laminating = [page for page in full_page if page not in laminating]

In [198]:
print(laminating)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 37, 38, 39, 40, 46, 47, 48, 49, 50, 51, 52, 61, 62, 63, 64]


In [199]:
print(non_laminating)

[11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 41, 42, 43, 44, 45, 53, 54, 55, 56, 57, 58, 59, 60, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100]


In [200]:
len(laminating)

25

In [201]:
len(non_laminating)

75

In [202]:
print("count | halaman yang dilaminating")
for i, l in enumerate(laminating):
    print(i+1, '  ', l)

count | halaman yang dilaminating
1    1
2    2
3    3
4    4
5    5
6    6
7    7
8    8
9    9
10    10
11    37
12    38
13    39
14    40
15    46
16    47
17    48
18    49
19    50
20    51
21    52
22    61
23    62
24    63
25    64


In [203]:
print("count | halaman yang tidak dilaminating")
expected = non_laminating[0]
start_range = None
for i, l in enumerate(non_laminating):
    l = int(l)
    # print(l)
    if l == expected:
        if start_range == None:
            start_range = l
        elif start_range != None:
            end_range = l
    elif l != expected:
        if start_range != None:
            # print(f'exp:{expected}')
            print(f'{start_range}-{end_range}')
            start_range = l
        elif start_range == None:
            print(l)
    if i == len(non_laminating) - 1:
        # print(f'exp:{expected}')
        print(f'{start_range}-{end_range}')
            
    expected = l + 1
        
    # print(i+1, '  ', l)
    # expected = l + 1

count | halaman yang tidak dilaminating
11-36
41-45
53-60
65-100


### Try to train model and verify whether it could provide nan results or not.

In [1]:
import coremlv2 as core
import importlib
importlib.reload(core)
core.os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
core._init_ml()

In [9]:
import coremlv2 as core; core._init_ml(); core.os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

core.safe_dataset_analyzer(db_ver='8', DB_ROOT_PATH='J:\#PROJECT\idx', model_no='321', start_from='vcn') # deployed
core.safe_dataset_analyzer(db_ver='8', DB_ROOT_PATH='J:\#PROJECT\idx', model_no='322', start_from='vcn') # deployed
core.safe_dataset_analyzer(db_ver='8', DB_ROOT_PATH='J:\#PROJECT\idx', model_no='323', start_from='vcn') # deployed
core.safe_dataset_analyzer(db_ver='8', DB_ROOT_PATH='J:\#PROJECT\idx', model_no='324', start_from='vcn') # deployed
core.safe_dataset_analyzer(db_ver='8', DB_ROOT_PATH='J:\#PROJECT\idx', model_no='325', start_from='vcn') # deployed

# Broadcast result to other model (focus on 321)
core.safe_dataset_analyzer(db_ver='8', DB_ROOT_PATH='J:\#PROJECT\idx', model_no='321', start_from='vde', stop_at='vfr')
core.safe_dataset_analyzer(db_ver='8', DB_ROOT_PATH='J:\#PROJECT\idx', model_no='321', start_from='vfr', stop_at='vid')
core.safe_dataset_analyzer(db_ver='8', DB_ROOT_PATH='J:\#PROJECT\idx', model_no='321', start_from='vid', stop_at='vis')
core.safe_dataset_analyzer(db_ver='8', DB_ROOT_PATH='J:\#PROJECT\idx', model_no='321', start_from='vis', stop_at='vmx')
core.safe_dataset_analyzer(db_ver='8', DB_ROOT_PATH='J:\#PROJECT\idx', model_no='321', start_from='vmx', stop_at='vnz')
core.safe_dataset_analyzer(db_ver='8', DB_ROOT_PATH='J:\#PROJECT\idx', model_no='321', start_from='vnz', stop_at='vru')
core.safe_dataset_analyzer(db_ver='8', DB_ROOT_PATH='J:\#PROJECT\idx', model_no='321', start_from='vru', stop_at='vtr')
core.safe_dataset_analyzer(db_ver='8', DB_ROOT_PATH='J:\#PROJECT\idx', model_no='321', start_from='vtr', stop_at=None)

# Analyze only certain stocks that throw nan_loss either in loss / val_loss
# Use G: as DB_ROOT_PATH to avoid interference with ongoing dataset nan_loss scan in J:
core.safe_dataset_analyzer(db_ver='8', DB_ROOT_PATH='J:\#PROJECT\idx', model_no='326', analyze_only='vbr') # finished
core.safe_dataset_analyzer(db_ver='8', DB_ROOT_PATH='J:\#PROJECT\idx', model_no='326', analyze_only='vid') # finished
core.safe_dataset_analyzer(db_ver='8', DB_ROOT_PATH='J:\#PROJECT\idx', model_no='326', analyze_only='vcn') # finished
core.safe_dataset_analyzer(db_ver='8', DB_ROOT_PATH='J:\#PROJECT\idx', model_no='326', analyze_only='vde') # finished #
core.safe_dataset_analyzer(db_ver='8', DB_ROOT_PATH='J:\#PROJECT\idx', model_no='326', analyze_only='vdk') # finished
core.safe_dataset_analyzer(db_ver='8', DB_ROOT_PATH='J:\#PROJECT\idx', model_no='326', analyze_only='ves') # finished
core.safe_dataset_analyzer(db_ver='8', DB_ROOT_PATH='J:\#PROJECT\idx', model_no='326', analyze_only='vfr') # finished
core.safe_dataset_analyzer(db_ver='8', DB_ROOT_PATH='J:\#PROJECT\idx', model_no='326', analyze_only='vgb') # finished
core.safe_dataset_analyzer(db_ver='8', DB_ROOT_PATH='J:\#PROJECT\idx', model_no='326', analyze_only='vgr') # finished
core.safe_dataset_analyzer(db_ver='8', DB_ROOT_PATH='J:\#PROJECT\idx', model_no='326', analyze_only='vhk') # finished
core.safe_dataset_analyzer(db_ver='8', DB_ROOT_PATH='J:\#PROJECT\idx', model_no='326', analyze_only='vus') # finished
core.safe_dataset_analyzer(db_ver='8', DB_ROOT_PATH='J:\#PROJECT\idx', model_no='326', analyze_only='vil') # finished

# vau had no eligible tickers
core.safe_dataset_analyzer(db_ver='8', DB_ROOT_PATH='J:\#PROJECT\idx', model_no='326', analyze_only='vtw') # finished
core.safe_dataset_analyzer(db_ver='8', DB_ROOT_PATH='J:\#PROJECT\idx', model_no='326', analyze_only='vau') # skipped--
core.safe_dataset_analyzer(db_ver='8', DB_ROOT_PATH='J:\#PROJECT\idx', model_no='326', analyze_only='vat') # finished
core.safe_dataset_analyzer(db_ver='8', DB_ROOT_PATH='J:\#PROJECT\idx', model_no='326', analyze_only='vbr') # finished
core.safe_dataset_analyzer(db_ver='8', DB_ROOT_PATH='J:\#PROJECT\idx', model_no='326', analyze_only='vin') # finished
core.safe_dataset_analyzer(db_ver='8', DB_ROOT_PATH='J:\#PROJECT\idx', model_no='326', analyze_only='vkr') # finished
core.safe_dataset_analyzer(db_ver='8', DB_ROOT_PATH='J:\#PROJECT\idx', model_no='326', analyze_only='vjp') # finished

# Speculative analysis
core.safe_dataset_analyzer(db_ver='8', DB_ROOT_PATH='J:\#PROJECT\idx', model_no='326', analyze_only='vlv') # finished
core.safe_dataset_analyzer(db_ver='8', DB_ROOT_PATH='J:\#PROJECT\idx', model_no='326', analyze_only='vmx') # finished
core.safe_dataset_analyzer(db_ver='8', DB_ROOT_PATH='J:\#PROJECT\idx', model_no='326', analyze_only='vmy') # finished
core.safe_dataset_analyzer(db_ver='8', DB_ROOT_PATH='J:\#PROJECT\idx', model_no='326', analyze_only='vnl') # finished
core.safe_dataset_analyzer(db_ver='8', DB_ROOT_PATH='J:\#PROJECT\idx', model_no='326', analyze_only='vno') # finished
core.safe_dataset_analyzer(db_ver='8', DB_ROOT_PATH='J:\#PROJECT\idx', model_no='326', analyze_only='vnz') # finished
core.safe_dataset_analyzer(db_ver='8', DB_ROOT_PATH='J:\#PROJECT\idx', model_no='326', analyze_only='vpl') # finished
core.safe_dataset_analyzer(db_ver='8', DB_ROOT_PATH='J:\#PROJECT\idx', model_no='326', analyze_only='vpt') # finished
core.safe_dataset_analyzer(db_ver='8', DB_ROOT_PATH='J:\#PROJECT\idx', model_no='326', analyze_only='vqa') # finished
core.safe_dataset_analyzer(db_ver='8', DB_ROOT_PATH='J:\#PROJECT\idx', model_no='326', analyze_only='vru') # finished
core.safe_dataset_analyzer(db_ver='8', DB_ROOT_PATH='J:\#PROJECT\idx', model_no='326', analyze_only='vse') # finished
core.safe_dataset_analyzer(db_ver='8', DB_ROOT_PATH='J:\#PROJECT\idx', model_no='326', analyze_only='vsg') # finished
core.safe_dataset_analyzer(db_ver='8', DB_ROOT_PATH='J:\#PROJECT\idx', model_no='326', analyze_only='vth') # finished
core.safe_dataset_analyzer(db_ver='8', DB_ROOT_PATH='J:\#PROJECT\idx', model_no='326', analyze_only='vtr') # finished

# Unavailable nan_loss
core.safe_dataset_analyzer(db_ver='8', DB_ROOT_PATH='J:\#PROJECT\idx', model_no='326', analyze_only='var')
core.safe_dataset_analyzer(db_ver='8', DB_ROOT_PATH='J:\#PROJECT\idx', model_no='326', analyze_only='vbe')
core.safe_dataset_analyzer(db_ver='8', DB_ROOT_PATH='J:\#PROJECT\idx', model_no='326', analyze_only='vch')
core.safe_dataset_analyzer(db_ver='8', DB_ROOT_PATH='J:\#PROJECT\idx', model_no='326', analyze_only='vcz')
core.safe_dataset_analyzer(db_ver='8', DB_ROOT_PATH='J:\#PROJECT\idx', model_no='326', analyze_only='vfi')
core.safe_dataset_analyzer(db_ver='8', DB_ROOT_PATH='J:\#PROJECT\idx', model_no='326', analyze_only='vie')
core.safe_dataset_analyzer(db_ver='8', DB_ROOT_PATH='J:\#PROJECT\idx', model_no='326', analyze_only='vis')

# New model analysis
core.safe_dataset_analyzer(db_ver='8', DB_ROOT_PATH='J:\#PROJECT\idx', model_no='326', analyze_only='vid') # finished

In [7]:
# Run simple model test
gen = core.load_dataset_wsd_traintest(subset='validation', dataset_size='small', ROOT_PATH='H:\#PROJECT\idx', db_ver='8', batch_size=128, shuffle_buffer_size=1024, seed=0, generator=False)

internal_file_path = core.os.path.join(dataset_path, 'vid')
internal_file_path = core.os.path.join(internal_file_path, 'ACES.JK_shift0_interval5_recurrent120_split0.8.hdf5')

with core.h5py.File(internal_file_path, 'r') as f:
    val_inputs, val_labels = f['train_inputs'][()], f['train_labels'][()]
    inputs, labels = f['test_inputs'][()], f['test_labels'][()]

# Run model
model = core.sample_model_v1()
history = model.fit(inputs, labels, validation_data=(val_inputs, val_labels), epochs=1, verbose=1)

Total constituents: 12


### Check composition of nan_loss to all available tickers
It turns out that all dataset produce nan_loss. Which part that it turns odd?

In [7]:
import coremlv2 as core
import importlib
importlib.reload(core)

<module 'coremlv2' from 'C:\\Users\\Jonathan Raditya\\documents\\#PROJECT\\idx\\coremlv2.py'>

In [8]:
db_ver = '8'
DB_ROOT_PATH = 'H:\#PROJECT\idx'
vabbrs = core.extract_dataset_vabbrs()
for vabbr in vabbrs:
    available_tickers = core.safe_dataset_list(vabbr, db_ver=db_ver, DB_ROOT_PATH=DB_ROOT_PATH)
    print(f'{vabbr}: {len(available_tickers)}')

var: 61
vat: 111
vbe: 116
vbr: 395
vch: 89
vcn: 1809
vcz: 20
vde: 4233
vdk: 116
ves: 133
vfi: 154
vfr: 572
vgb: 1760
vgr: 83
vhk: 1863
vid: 419
vie: 29
vil: 288
vin: 3698
vis: 15
vjp: 2488
vkr: 1550
vlv: 7
vmx: 279
vmy: 757
vnl: 70
vno: 186
vnz: 106
vpl: 327
vpt: 32
vqa: 38
vru: 203
vse: 616
vsg: 482
vth: 507
vtr: 369
vtw: 1685
vus: 4409


### Check load data shape

In [30]:
# !pip install --upgrade tensorflow --quiet
# !pip install keras_tuner --quiet
# !pip install tensorflow-io --quiet
# # Google colab modules
# from google.colab import drive
import sys, importlib

# # Mount drive
# drive.mount('/content/gdrive', force_remount=True)
ROOT_PATH = './'
# sys.path.append(ROOT_PATH)

import coremlv2 as core
core._init_ml()
# core._init_models()
# core.os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

# Reload coreml
importlib.reload(core)
import keras_tuner as kt

In [31]:
# RERUN WITH REVISED traintest slice code
model_no = '326'
kt_iter = '33'
ticker_group = ['wsd']
epochs = 40
max_epochs = 40
batch_size = 64

dataset_size = 'half_new_wsd'
shuffle_buffer_size = 2048
generator = False

train_inputs, train_labels = core.load_dataset_wsd_traintest(subset='training', dataset_size=dataset_size, ROOT_PATH='''J:\#PROJECT\idx''', db_ver='8', batch_size=batch_size, shuffle_buffer_size=shuffle_buffer_size, seed=0, generator=generator, model_no=model_no)

validation_inputs, validation_labels = core.load_dataset_wsd_traintest(subset='validation', dataset_size=dataset_size, ROOT_PATH='''J:\#PROJECT\idx''', db_ver='8', batch_size=batch_size, shuffle_buffer_size=shuffle_buffer_size, seed=0, generator=generator, model_no=model_no)

Total constituents: 42
Total constituents: 42


In [32]:
train_inputs.shape

(99531, 120, 60)

In [33]:
validation_inputs.shape

(18686, 120, 60)

### Check nan_loss availability

In [1]:
import coremlv2 as core
import importlib
importlib.reload(core)

<module 'coremlv2' from 'C:\\Users\\Jonathan Raditya\\documents\\#PROJECT\\idx\\coremlv2.py'>

In [17]:
db_path = 'J:\\#PROJECT\\idx\\db\\v8\\dataset'
current_model_no = '326'
benchmark_model_no = '321'
db_path_current = core.os.path.join(db_path, f'nan_loss_{current_model_no}')
db_path_benchmark = core.os.path.join(db_path, f'nan_loss_{benchmark_model_no}')

In [18]:
current_content = core.os.listdir(db_path_current)
benchmark_content = core.os.listdir(db_path_benchmark)
unavailable_slice = [content.split('_')[0] for content in benchmark_content if content not in current_content]

In [19]:
unavailable_slice

['var', 'vbe', 'vch', 'vcz', 'vfi', 'vie', 'vis']

### Recreate generator so it can be used in NAS
- Create simple modified function and NAS with 5 chars random kt_iter version
- use "small" dataset_size
- max_epochs = 2
- test if NAS can be continued to next iteration

`env:tensorflow`

In [1]:
# Load necessary modules
import importlib
ROOT_PATH = './'

import coremlv2 as core
core._init_ml()
# core.os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

# Reload coreml
importlib.reload(core)
import keras_tuner as kt

In [2]:
# Limiting GPU memory growth
gpus = core.tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            core.tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = core.tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs, ", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

1 Physical GPUs,  1 Logical GPUs


In [11]:
# Use generator
from tensorflow.keras.utils import Sequence
import numpy as np
class DataGenerator(Sequence):
    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size
    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))
    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        return batch_x, batch_y     

#### Create custom hdf5 pipeline

In [3]:
def load_dataset_wsd_traintest(subset, dataset_size, ROOT_PATH='''H:\#PROJECT\idx''', db_ver='8', batch_size=128, shuffle_buffer_size=8192, seed=0, generator=False, model_no='sample1'):
    '''
    subset: `training`, `validation`
    dataset_size: `small`, `medium`, `large`
    
    use env:tensorflow.
    '''
    dataset_size_dict = {'xsmall':
                         {'training':[True,0.0005,0.005],
                          'validation':[False,0.00025,0.0025]},
                         'small':
                         {'training':[True,0.001,0.01],
                          'validation':[False,0.0005,0.005]},
                         'medium':
                         {'training':[True,0.1,0.4],
                          'validation':[False,0,1]},
                         'large':
                         {'training':[True,0.2,0.8],
                          'validation':[False,0,1]},
                         'extralarge':
                         {'training':[True,0.5,1],
                          'validation':[False,0,1]},
                         'exploration':
                         {'training':[True,1,1],
                          'validation':[False,1,1]},
                         'full':
                         {'training':[True,0.4,0.8],
                          'validation':[False,0,0.2]},
                         'half_new_wsd':
                         {'training':[True,0,0.1],
                          'validation':[False,0,0.1]},
                         'full_new_wsd':
                         {'training':[True,0,1],
                          'validation':[False,0,1]},
                         'full_new_wsd_mix':
                         {'training':[True,0.005,1],
                          'validation':[False,0,1]},
                         'full_new_wsd_mix1':
                         {'training':[True,0.01,1],
                          'validation':[False,0,1]},
                         'full_new_wsd_mix2':
                         {'training':[True,0.02,1],
                          'validation':[False,0,1]},}
    slice_from_beginning = dataset_size_dict[dataset_size][subset][0]
    constituent_limits = dataset_size_dict[dataset_size][subset][1]
    id_constituent = dataset_size_dict[dataset_size][subset][2]
    
    return load_dataset_wsd(ROOT_PATH=ROOT_PATH, db_ver=db_ver, constituent_limits=constituent_limits, id_constituent=id_constituent, batch_size=batch_size, shuffle_buffer_size=shuffle_buffer_size, seed=seed, slice_from_beginning=slice_from_beginning, generator=generator, model_no=model_no)

def load_dataset_wsd(ROOT_PATH='''H:\#PROJECT\idx''', db_ver='8', constituent_limits=0.01, id_constituent=0.1, batch_size=128, shuffle_buffer_size=8192, seed=0, slice_from_beginning=True, generator=True, model_no='sample1'):
    '''Load random world stock database constituent
    to make up training database as generator.
    
    /test_inputs are used to make sure whole
    dataset cover most of abstraction from various
    tickers & stock markets (smaller constituent
    compared to /train_inputs)
    ex
    Try:
        (x) 1. remove shuffle constituent to prevent
            stopiteration (aborted.)
            
        (v) 2. If vid is being looped and `slice_from_beginning` == True
            (indicating vid for training set), load from /train_inputs
            and /train_labels.
            
            Otherwise (`slice_from_beginning` == False) while vid being looped,
            load from /test_inputs and /test_labels.
    
    use env:tensorflow.
    
    #### Run log
    0. Default state from coremlv2
    1. Move important functions to randomtry and construct pipeline. **Result**: Run and error after 1st iteration, as expected.
    2. Add xsmall dataset_size for faster iteration, comment random.seed & random.shuffle to test if shuffling can cause error. **Result**: Stop iteration error.
    3. Uncomment random.seed & random.shuffle. Use conventional h5py read and then pass it to DataGenerator class if generator. Continue as usual to next section. **Result**: Stop iteration error.
    4. Try itertools.repeat. **Result**: Attribute error, no shape attribute.
    5. Try itertools.cycle. **Result**: Infinite loop (never ending 1st epoch)
    6. Restore to 0 state, try .concatenate method to chaining multiple IODataset. **Result**: SUCCESS!
    7. Try using full_new_wsd to test if any memory concern occurs. **Result**: SUCCESS!. Fast dataset loading, shuffling, and able to continue to next iteration. No memory problem occurs.
    8. Try using GPU, 5 epochs. **Result**: Strange result, the process finish so fast although all idx data were used. Turns out that .concatenate returns a modified object. 
    9. Modify `inputs = inputs.concatenate(next_inputs)`. **Result**: SUCCESS! Implemented version.

    #### Additional info:
    - full_new_wsd with model 327 only occupy 400mb memory. Shiet! wkwk
    - next potential problem: how to increase gpu utilization and direct storage read speed.
    - try different batch_size to see if overall speed is increased
        Using 327/full_new_wsd
        2048 -> , 2.2gb
        512 -> 321ms/step, 536sec, 1.4gb
        128 -> 96ms/step, 641sec, 400mb
        64 -> 46ms/step, 726sec, 200mb
    - more batch_size -> same gpu utilization but with higher board power draw & gpu clock (gpu-z)
    - kt explore 327/34/full_new_wsd_mix terakhir lupa diganti hypermodelnya, masih model_326_kt. Di re-run aja
    '''
    # Construct dataset_path
    dataset_path = core.os.path.join(ROOT_PATH, 'db')
    dataset_path = core.os.path.join(dataset_path, f'v{db_ver}')
    dataset_path = core.os.path.join(dataset_path, 'dataset')
    
    if not core.os.path.exists(dataset_path):
        raise ValueError(f'Dataset path not exists: {dataset_path}')
    elif core.os.path.exists(dataset_path):
        dataset_list = core.extract_dataset_vabbrs(db_ver=db_ver, DB_ROOT_PATH=ROOT_PATH)
        # dataset_list = os.listdir(dataset_path)
        
    # Fetch constituent list from database
    file_list = {}
    for dataset in dataset_list:
        file_list[dataset] = core.safe_dataset_list(dataset, db_ver=db_ver, DB_ROOT_PATH=ROOT_PATH, return_full_filename=True, model_no=model_no)
        # dataset_ver_path = os.path.join(dataset_path, dataset)
        # if os.path.isdir(dataset_ver_path):
        #     dataset_ver_files = os.listdir(dataset_ver_path)
        #     file_list[dataset] = dataset_ver_files
        # elif os.path.isfile(dataset_ver_path):
        #     pass

    # Select constituent based on consitituent_limits
    '''
    constituent_limits: percentage of different stocks that
        make up the dataset
    '''
    constituent_list = {}
    for file_list_key in file_list:
        file = file_list[file_list_key]
        if file_list_key == 'vid':
            slicer = int(len(file) * id_constituent)
        elif file_list_key != 'vid':
            slicer = int(len(file) * constituent_limits)
        if slicer < 1:
            continue
        elif slicer >= 1:
            if slice_from_beginning:
                constituent_list[file_list_key] = file[:slicer]
            elif not slice_from_beginning:
                constituent_list[file_list_key] = file[-slicer:]
   
    # Load from /test_inputs & /test_labels
    for i, constituent_list_key in enumerate(constituent_list):
        # `constituent_list_key`: vid, vus, etc.
        constituents = constituent_list[constituent_list_key]
        constituent_dir_path = core.os.path.join(dataset_path, constituent_list_key)
        for j, constituent in enumerate(constituents):
            # Full file path
            constituent_file_path = core.os.path.join(constituent_dir_path, constituent)
            if generator:
                pass
#                 if (constituent_list_key == 'vid') and slice_from_beginning:
#                     # vid, training
#                     inputs = core.tfio.IODataset.from_hdf5(constituent_file_path, '/train_inputs')
#                     labels = core.tfio.IODataset.from_hdf5(constituent_file_path, '/train_labels')
#                 elif ((constituent_list_key == 'vid') and not slice_from_beginning) or (constituent_list_key != 'vid'):
#                     # vid, testing
#                     # another stock market for train/testing
#                     inputs = core.tfio.IODataset.from_hdf5(constituent_file_path, '/test_inputs')
#                     labels = core.tfio.IODataset.from_hdf5(constituent_file_path, '/test_labels')

#                 # Zip inputs and labels
#                 gen = core.tf.data.Dataset.zip((inputs, labels))

#                 # Apply batches
#                 gen = gen.batch(batch_size)

#                 # Apply shuffle
#                 gen = gen.shuffle(buffer_size=shuffle_buffer_size)
            elif not generator:
                if (constituent_list_key == 'vid') and slice_from_beginning:
                    # vid, training
                    with core.h5py.File(constituent_file_path, 'r') as f:
                        inputs = f['train_inputs'][()]
                        labels = f['train_labels'][()]
                elif ((constituent_list_key == 'vid') and not slice_from_beginning) or (constituent_list_key != 'vid'):
                    # vid, testing
                    # another stock market for train/testing
                    with core.h5py.File(constituent_file_path, 'r') as f:
                        inputs = f['test_inputs'][()]
                        labels = f['test_labels'][()]
                
            # if generator:
            #     gen = DataGenerator(inputs, labels, batch_size)
            # elif not generator:
            #     pass

            if i + j == 0:
                gen_counter = 0
                if generator:
                    # combined_gen = [gen]
                    
                    if (constituent_list_key == 'vid') and slice_from_beginning:
                        # vid, training
                        inputs = core.tfio.IODataset.from_hdf5(constituent_file_path, '/train_inputs')
                        labels = core.tfio.IODataset.from_hdf5(constituent_file_path, '/train_labels')
                    elif ((constituent_list_key == 'vid') and not slice_from_beginning) or (constituent_list_key != 'vid'):
                        # vid, testing
                        # another stock market for train/testing
                        inputs = core.tfio.IODataset.from_hdf5(constituent_file_path, '/test_inputs')
                        labels = core.tfio.IODataset.from_hdf5(constituent_file_path, '/test_labels')
                    
                elif not generator:
                    # print(inputs.shape)
                    combined_inputs = inputs.copy()
                    combined_labels = labels.copy()
                    constituents_count = 1
            elif i + j > 0:
                gen_counter += 1
                if generator:
                    # combined_gen.append(gen)
                    
                    if (constituent_list_key == 'vid') and slice_from_beginning:
                        # vid, training
                        next_inputs = core.tfio.IODataset.from_hdf5(constituent_file_path, '/train_inputs')
                        next_labels = core.tfio.IODataset.from_hdf5(constituent_file_path, '/train_labels')
                    elif ((constituent_list_key == 'vid') and not slice_from_beginning) or (constituent_list_key != 'vid'):
                        # vid, testing
                        # another stock market for train/testing
                        next_inputs = core.tfio.IODataset.from_hdf5(constituent_file_path, '/test_inputs')
                        next_labels = core.tfio.IODataset.from_hdf5(constituent_file_path, '/test_labels')
                        
                    inputs = inputs.concatenate(next_inputs)
                    labels = labels.concatenate(next_labels)
                    
                elif not generator:
                    combined_inputs = core.np.vstack((combined_inputs, inputs))
                    combined_labels = core.np.vstack((combined_labels, labels))
                    constituents_count += 1

    if generator:
        # print(f'Total constituents: {len(combined_gen)}')
        # Shuffle constituent
        # core.random.seed(seed)
        # core.random.shuffle(combined_gen)
        # combined_gen = core.itertools.chain(*combined_gen)
        # return combined_gen
        # return core.itertools.repeat(combined_gen, 2)
        # return core.itertools.cycle(combined_gen)
        
        # New framework using .concatenate method
        print(f'Total constituents: {gen_counter}')
        # Zip inputs and labels
        gen = core.tf.data.Dataset.zip((inputs, labels))

        # Apply batches
        gen = gen.batch(batch_size)

        # Apply shuffle
        gen = gen.shuffle(buffer_size=shuffle_buffer_size)
        return gen
    elif not generator:
        print(f'Total constituents: {constituents_count}')
        core.np.random.seed(seed)
        rand_index = core.np.random.permutation(len(combined_inputs))
        combined_inputs, combined_labels = combined_inputs[rand_index], combined_labels[rand_index]
        return combined_inputs, combined_labels

#### Run log
0. Default state from coremlv2
1. Move important functions to randomtry and construct pipeline. **Result**: Run and error after 1st iteration, as expected.
2. Add xsmall dataset_size for faster iteration, comment random.seed & random.shuffle to test if shuffling can cause error. **Result**: Stop iteration error.
3. Uncomment random.seed & random.shuffle. Use conventional h5py read and then pass it to DataGenerator class if generator. Continue as usual to next section. **Result**: Stop iteration error.
4. Try itertools.repeat. **Result**: Attribute error, no shape attribute.
5. Try itertools.cycle. **Result**: Infinite loop (never ending 1st epoch)
6. Restore to 0 state, try .concatenate method to chaining multiple IODataset. **Result**: SUCCESS!
7. Try using full_new_wsd to test if any memory concern occurs. **Result**: SUCCESS!. Fast dataset loading, shuffling, and able to continue to next iteration. No memory problem occurs.
8. Try using GPU, 5 epochs. **Result**: Strange result, the process finish so fast although all idx data were used. Turns out that .concatenate returns a modified object. 
9. Modify `inputs = inputs.concatenate(next_inputs)`. **Result**: SUCCESS! Implemented version.

#### Additional info:
- full_new_wsd with model 327 only occupy 400mb memory. Shiet! wkwk
- next potential problem: how to increase gpu utilization and direct storage read speed.
- try different batch_size to see if overall speed is increased
    Using 327/full_new_wsd
    2048 -> , 2.2gb
    512 -> 321ms/step, 536sec, 1.4gb
    128 -> 96ms/step, 641sec, 400mb
    64 -> 46ms/step, 726sec, 200mb
- more batch_size -> same gpu utilization but with higher board power draw & gpu clock (gpu-z)
- kt explore 327/34/full_new_wsd_mix terakhir lupa diganti hypermodelnya, masih model_326_kt. Di re-run aja


In [4]:
model_no = '327'
kt_iter = 'try9'
ticker_group = ['wsd']
epochs = 5
max_epochs = 5
batch_size = 2048

dataset_size = 'full_new_wsd'
shuffle_buffer_size = 10
generator = True

train_gen = load_dataset_wsd_traintest(subset='training', dataset_size=dataset_size, ROOT_PATH='''J:\#PROJECT\idx''', db_ver='8', batch_size=batch_size, shuffle_buffer_size=shuffle_buffer_size, seed=0, generator=generator, model_no=model_no)

validation_gen = load_dataset_wsd_traintest(subset='validation', dataset_size=dataset_size, ROOT_PATH='''J:\#PROJECT\idx''', db_ver='8', batch_size=batch_size, shuffle_buffer_size=shuffle_buffer_size, seed=0, generator=generator, model_no=model_no)

Total constituents: 422
Total constituents: 422


In [ ]:
tuner = kt.Hyperband(hypermodel=core.model_327_kt, objective='val_loss', max_epochs=max_epochs, hyperband_iterations=1, overwrite=True, directory=f'{ROOT_PATH}models/kt/v{kt_iter}/', project_name='_'.join(ticker_group))

tuner.search(train_gen, validation_data=validation_gen, epochs=epochs, verbose=1)

Trial 6 Complete [00h 27m 10s]
val_loss: 0.6478671431541443

Best val_loss So Far: 0.6478671431541443
Total elapsed time: 01h 59m 57s

Search: Running Trial #7

Hyperparameter    |Value             |Best Value So Far 
lr                |0.00084556        |0.0027236         
r_units_1         |96                |48                
r_units_2         |64                |32                
d_units_1         |96                |80                
d_units_2         |64                |112               
tuner/epochs      |5                 |5                 
tuner/initial_e...|2                 |2                 
tuner/bracket     |1                 |1                 
tuner/round       |1                 |1                 
tuner/trial_id    |20e7c70ed815b8c...|fd080aa3e84c365...

Epoch 3/5
    413/Unknown - 362s 845ms/step - loss: 0.6742 - accuracy: 0.5840

#### Hitmat assistant

In [1]:
import time
import datetime

In [2]:
time.now()

AttributeError: module 'time' has no attribute 'now'

In [4]:
int(datetime.datetime.now().strftime('%H'))

15

In [17]:
def hour_truth_map(start, stop):
    # Truth mapper generator
    start = 20
    stop = 4

    # start = 2
    # stop = 17
    assert start != stop, 'Enter different value for start and stop'
    direction = stop - start
    if direction < 0:
        # Over midnight
        before_midnight = [start + i for i in range(24 - start)]
        after_midnight = [i for i in range(stop)]
        truth_map = before_midnight + after_midnight
    elif direction > 0:
        # Not over midnight
        truth_map = [start + i for i in range(stop - start)]
    return truth_map

In [18]:
truth_map

[20, 21, 22, 23, 0, 1, 2, 3]

In [19]:
a = [1,2,3,4,5]

In [22]:
a[:10]

[1, 2, 3, 4, 5]

In [10]:
core.pd.Timestamp(1654214400000000000)

Timestamp('2022-06-03 00:00:00')

In [6]:
a = [1,2,3,4,5,6]
a[-3:]

[4, 5, 6]